### Importação de Bibliotecas Necessárias

In [ ]:
library(tseries) 
library(tidyverse)
library(dplyr)
library(readxl)
library("TTR")
library(forecast)
library(lmtest)
library(FitAR)
library(randtests)
library(seasonalview)
library(moments)

### Tema de alguns gráficos

In [ ]:
tema = theme(panel.background = element_rect(fill='black'),
        panel.grid =element_blank(),
        plot.background = element_rect(fill='black'),
        axis.text = element_text(colour='white',size=10), 
        #panel.grid.major.x = element_line(colour='grey60',linetype = 4,size = 0.2),
        panel.grid.major.y = element_line(colour='white',linetype = 1,size = 0.1),
        axis.line = element_line(colour='white'),
        axis.title=element_text(colour='white'),
        plot.title = element_text(colour='white'))

### Leitura do Dataset

In [ ]:

df <-
  read.csv(
    'dataset/preco.csv',
    sep = "," ,
    dec = ',',
    header = T,
    stringsAsFactors = FALSE
  )




### Transformar em Série Temporal

In [ ]:
df_series = ts(df, start=c(2013,1), frequency = 12)
df_series

### Plotar o gráfico da Série Temporal - Percebemos que visualmente aparenta não ser estacionária 

In [ ]:
plot(df_series,  main="Série Temporal", ylab="Preço do Petróleo", col="#DA70D6", lty="dashed")

## ADF é o teste de Dickey

### Teste de estacionariedade de Dickey-Fuller Aumentado (Dickey & Fuller, 1979) , com as seguintes hipóteses: 
### H0 : a série não é estacionária 
### H1 : a série é estacionária 
### Regra de decisão: Quando o nível descritivo é < 0,05 rejeitamos H0 , ou seja, há evidência de que a série é estacionária.


### Realizar o Teste de Estacionariedade - Como o p-value é maior que 0.05, portanto a série não é estacionária.

### O Resultado deu p-value = 0.7328. Portanto, é maior que 0.05, logo não é estacionária

In [ ]:
adf.test(df_series)

### Realização da primeira diferenciação para transformar a série em estacionária

In [ ]:
serie_dif1 = diff(df_series)
plot(serie_dif1)

## ADF é o teste de Dickey

### Teste de estacionariedade de Dickey-Fuller Aumentado (Dickey & Fuller, 1979) , com as seguintes hipóteses: 
### H0 : a série não é estacionária 
### H1 : a série é estacionária 
### Regra de decisão: Quando o nível descritivo é < 0,05 rejeitamos H0 , ou seja, há evidência de que a série é estacionária.


### Verificar se a Série se tornou estacionária com a primeira diferenciação. 

# Pode notar que o resultado do p-value é 0.07,ou seja, 0.07 > 0.05. Logo, não é estacionária.


In [ ]:
adf.test(serie_dif1)

### Realizar a segunda diferenciação para que a série fique estacionária

In [ ]:
serie_dif2 = diff(serie_dif1)

## ADF é o teste de Dickey

### Teste de estacionariedade de Dickey-Fuller Aumentado (Dickey & Fuller, 1979) , com as seguintes hipóteses: 
### H0 : a série não é estacionária 
### H1 : a série é estacionária 
### Regra de decisão: Quando o nível descritivo é < 0,05 rejeitamos H0 , ou seja, há evidência de que a série é estacionária.


### Observa-se que o p-value (0.01) < 0.05. Logo, a série é estacionária para esse nível descritivo

In [ ]:
adf.test(serie_dif2)

### Gráfico da Série Temporal Diferenciada

In [ ]:
plot(serie_dif2, type="o", lty="dashed",main="Série Temporal",col="#1E90FF")

# Com a série estácionaria conseguimos realizar o modelo do arima, sabemos que o i está relacionado com a diferenciação, logo i =2.

# Analisar os gráficos acf e pacf para verificar os outros parâmetros do Arima.



In [ ]:
par(mfrow=c(2,1), mar=c(4,4,4,1)+.1)
acf(serie_dif2)
pacf(serie_dif2)

### Separação de Treino e Teste

In [ ]:
##treino
df_series_treino <- window(df_series, start = 2013, end = c(2018,12))
##teste
df_series_test <- window(df_series,start = 2019)


# Modelo Arima - colocando os Parâmetros do Arima

In [ ]:
modelo1 = arima(df_series_treino,order=c(1,2,1))

### Verificação dos resíduos

In [ ]:
summary(modelo1)
tsdiag(modelo1)

### Verificação dos resíduos

In [ ]:
Box.test(residuals(modelo1),type="Ljung-Box")

### Realizar o forecasting

In [ ]:
forecasting=forecast::forecast(modelo1,h=10)
plot(forecasting)

In [ ]:
previsoes_arma121 = forecast::forecast(modelo1,h=8)

dados_prev = rbind(data.frame(serie = as.double(df_series)),
      data.frame(serie = previsoes_arma121$mean)) 

dados_prev = dados_prev %>% mutate(n = 1:95)

In [ ]:
dados_prev %>% ggplot(aes(x=n,y = serie))+
  geom_path(colour=ifelse(dados_prev$n<=104,'dodgerblue','firebrick'),linetype=1)+
          labs(title = 'Preços mensais do petróleo',
               x= 'Nº Mês', y ='Valor')+tema

In [ ]:
df_series_test = data.frame(n=1:length(df_series_test)[1], df_series_test)
df_series_treino = data.frame(n = 1:length(df_series_treino)[1], df_series_treino)
  ##geom_path(colour=ifelse(dados_prev$n<=104,'dodgerblue','firebrick'),linetype=1)+
preco %>% ggplot(aes(x=data,y = preco))+
  geom_path(colour='dodgerblue')+
  geom_path(data=df_series_test,aes(80:87,df_series_test),colour='red',alpha=0.4)+
  geom_path(data= dados_prev[87:94,],aes(x =87:94,y=dados_prev$serie[87:94] ), colour='red',linetype=2)+
  labs(title = 'Quantidade de solicitações de empréstimo semanal',
       x= 'Nº Semana', y ='Quantidade de Empréstimos')+tema